In [ ]:
# !pip install SQLAlchemy psycopg2-binary pandas python-dotenv
import os
import pandas as pd
from dotenv import load_dotenv
from sqlalchemy import create_engine

load_dotenv()
db_url = os.getenv("DATABASE_URL")
if not db_url:
    raise RuntimeError("Missing DATABASE_URL in .env")

# Chuẩn hoá prefix nếu .env dùng 'postgres://'
if db_url.startswith("postgres://"):
    db_url = db_url.replace("postgres://", "postgresql+psycopg2://", 1)

engine = create_engine(db_url, pool_pre_ping=True)

QUERY = """
SELECT *
FROM tiktok_trend_capture_detail
WHERE eer_score > 1
  AND (
        NULLIF(btrim(transcripts),  '') IS NOT NULL
     OR NULLIF(btrim(description), '') IS NOT NULL
      )
  AND duration <= 30
ORDER BY video_id ASC;
"""

df = pd.read_sql_query(QUERY, con=engine, index_col="video_id")
print(df.shape)
df.head(20)

(140, 14)


,playcount,sharecount,heartcount,savecount,eer_score,commentcount,description,hashtag,music_url,ranking,published_at,duration,transcripts,url
video_id,,,,,,,,,,,,,,
7530146465381879060,23100000,129800,417700,25287,6.97,6009,hay,"[blockblast, hungrystudio, blockblastlove]",https://www.tiktok.com/music/nhạc-nền-6-con-hà...,None,2025-07-22,27,,https://www.tiktok.com/@_/video/75301464653818...
7530960710038375700,12300000,19100,203900,8769,3.55,2239,,"[blockblast, hungrystudio, blockblastlove, fyp...",https://www.tiktok.com/music/nhạc-nền-original...,None,2025-07-24,7,oh no no no no no no no,https://www.tiktok.com/@_/video/75309607100383...
7535072898050985224,6900000,32900,218600,5826,7.61,3636,,[],https://www.tiktok.com/music/nhạc-nền-Linh-3-T...,None,2025-08-04,28,ăn Cho vừa thôi đang chén nữa à đâu SAO hông l...,https://www.tiktok.com/@_/video/75350728980509...
7537488053154204936,12700000,70300,116000,7182,5.46,2655,Cô chú mua cho bé chơi nhé,"[foryou, fyp, xuhuong, phongvanembe]",https://www.tiktok.com/music/nhạc-nền-ㅤ⋆౨ৎ𝓐𝓲⟡˖...,None,2025-08-11,8,con muốn mẹ mua Cho con gì hông con muốn cái đ...,https://www.tiktok.com/@_/video/75374880531542...
7538400798028451080,11800000,23800,775500,33315,10.99,4428,Lần đầu tập ngôn ngữ ký hiệu có gì chưa đúng m...,[thanhmeo18],https://www.tiktok.com/music/nhạc-nền-Còn-Gì-Đ...,None,2025-08-13,20,,https://www.tiktok.com/@_/video/75384007980284...
7538669201578560786,5400000,20200,84100,7966,5.89,2631,Pom orange,"[phocsoc, pomeranian, pomxinhsaigon, dog, pome...",https://www.tiktok.com/music/nhạc-nền-HANA-CẨM...,None,2025-08-14,17,Kia nói là anh tệ kệ mình nhắm mắt yêu luôn ng...,https://www.tiktok.com/@_/video/75386692015785...
7538786096793079054,157900000,8100000,18600000,1058466,54.67,88300,,"[motivation, fyp, foryou, viral, foryoupage]",https://www.tiktok.com/music/nhạc-nền-original...,None,2025-08-14,16,Không gì sánh bằng một kỳ nghỉ bằng máy bay ph...,https://www.tiktok.com/@_/video/75387860967930...
7539038956126571797,258400000,9300000,26100000,1797005,42.64,200000,开会了😂,[],https://www.tiktok.com/music/nhạc-nền-原创音乐-753...,None,2025-08-15,8,,https://www.tiktok.com/@_/video/75390389561265...
7539404376188800264,6100000,163500,262800,14851,25.70,2343,- 💔.,[],https://www.tiktok.com/music/nhạc-nền-nhatochu...,None,2025-08-16,15,,https://www.tiktok.com/@_/video/75394043761888...


In [2]:
import numpy as np

# Chuẩn hoá 2 cột về chuỗi, bỏ NaN và trim
desc = df['description'].fillna('').astype(str).str.strip()
tran = df['transcripts'].fillna('').astype(str).str.strip()

# Nếu cả hai đều có nội dung: nối bằng 2 dòng trống; nếu chỉ có 1 bên: dùng bên đó
text = np.where((desc != '') & (tran != ''), desc + '\n\n' + tran, desc + tran)

# DataFrame mới chỉ gồm cột 'text', index chính là 'id' (đã có từ truy vấn)
df2 = pd.DataFrame({
    'text': (desc + " " + tran).str.replace(r'\s+', ' ', regex=True).str.strip()
}, index=df.index).rename_axis('id')

print(df2.shape)
df2.head()


(140, 1)


,text
id,
7530146465381879060,hay
7530960710038375700,oh no no no no no no no
7535072898050985224,ăn Cho vừa thôi đang chén nữa à đâu SAO hông l...
7537488053154204936,Cô chú mua cho bé chơi nhé con muốn mẹ mua Cho...
7538400798028451080,Lần đầu tập ngôn ngữ ký hiệu có gì chưa đúng m...


In [3]:
# Clean: lowercase, bỏ URL, bỏ ký tự đặc biệt, bỏ underscore, gộp khoảng trắng
df2['text'] = (
    df2['text'].fillna('')
      .str.lower()
      .str.replace(r'https?://\S+|www\.\S+', ' ', regex=True)   # bỏ URL nếu có
      .str.replace(r'[^\w\s]', ' ', regex=True)                 # giữ chữ/số Unicode & khoảng trắng
      .str.replace(r'_', ' ', regex=True)                       # bỏ underscore
      .str.replace(r'\s+', ' ', regex=True)                     # gộp khoảng trắng
      .str.strip()
)

df2.head()

,text
id,
7530146465381879060,hay
7530960710038375700,oh no no no no no no no
7535072898050985224,ăn cho vừa thôi đang chén nữa à đâu sao hông l...
7537488053154204936,cô chú mua cho bé chơi nhé con muốn mẹ mua cho...
7538400798028451080,lần đầu tập ngôn ngữ ký hiệu có gì chưa đúng m...


In [4]:
import pandas as pd

# n-gram mặc định: 2..20 (đổi tùy ý)
def build_ngrams_df(df, nmin=2, nmax=20, distinct=True, drop_empty=True):
    """
    df: DataFrame có index là id, cột 'text' (đã clean/lowercase).
    Trả về DataFrame có cột: id, n, list_n_gram (list[str]).
    """
    if 'text' not in df.columns:
        raise ValueError("df phải có cột 'text'")

    def unique_preserve_order(iterable):
        seen = set()
        out = []
        for x in iterable:
            if x not in seen:
                seen.add(x)
                out.append(x)
        return out

    rows = []
    text_series = df['text'].fillna('').astype(str)
    for id_, text in text_series.items():
        toks = text.split()
        L = len(toks)
        for n in range(nmin, nmax + 1):
            if L < n:
                grams_list = []
            else:
                grams_iter = (" ".join(toks[i:i+n]) for i in range(L - n + 1))
                grams_list = unique_preserve_order(grams_iter) if distinct else list(grams_iter)
            rows.append((id_, n, grams_list))

    out = pd.DataFrame(rows, columns=['id', 'n', 'list_n_gram'])
    if drop_empty:
        out = out[out['list_n_gram'].map(bool)].reset_index(drop=True)
    return out


ngrams_df = build_ngrams_df(df2, nmin=2, nmax=20, distinct=True, drop_empty=True)
print(ngrams_df.shape)
ngrams_df.head(10)

(1715, 3)


,id,n,list_n_gram
0,7530960710038375700,2,"[oh no, no no]"
1,7530960710038375700,3,"[oh no no, no no no]"
2,7530960710038375700,4,"[oh no no no, no no no no]"
3,7530960710038375700,5,"[oh no no no no, no no no no no]"
4,7530960710038375700,6,"[oh no no no no no, no no no no no no]"
5,7530960710038375700,7,"[oh no no no no no no, no no no no no no no]"
6,7530960710038375700,8,[oh no no no no no no no]
7,7535072898050985224,2,"[ăn cho, cho vừa, vừa thôi, thôi đang, đang ch..."
8,7535072898050985224,3,"[ăn cho vừa, cho vừa thôi, vừa thôi đang, thôi..."
9,7535072898050985224,4,"[ăn cho vừa thôi, cho vừa thôi đang, vừa thôi ..."


In [5]:
# ngrams_df: cột ['id','n','list_n_gram'] (list_n_gram là list distinct theo từng id)

# 1) "explode" mỗi n-gram thành 1 dòng
exploded = (
    ngrams_df
      .explode('list_n_gram', ignore_index=False)            # giữ nguyên id
      .rename(columns={'list_n_gram': 'ngram'})
)

# 2) lọc rỗng & đảm bảo không đếm lặp trong cùng (id, n, ngram)
exploded = exploded[exploded['ngram'].notna() & (exploded['ngram'] != '')]
exploded = exploded.drop_duplicates(subset=['id', 'n', 'ngram'])

# 3) Gom theo (n, ngram) -> tập id chứa ngram đó
dups = (
    exploded.groupby(['n', 'ngram'])['id']
    .agg(lambda s: sorted(set(s)))            # list id duy nhất
    .reset_index()
    .rename(columns={'id': 'ids'})
    .assign(id_count=lambda d: d['ids'].str.len())
)

# 4) Chỉ giữ n-gram trùng giữa các id (>= 2 id)
dups = dups[dups['id_count'] >= 2].sort_values(['n', 'id_count'], ascending=[True, False]).reset_index(drop=True)

In [6]:
dups

,n,ngram,ids,id_count
0,2,có thể,"[7538786096793079054, 7539415572929400072, 754...",8
1,2,ba minh,"[7540708188795014417, 7540879122965318932, 754...",5
2,2,bây giờ,"[7538786096793079054, 7540271629344034066, 754...",5
3,2,người ta,"[7538669201578560786, 7540242465832389895, 754...",5
4,2,nhưng mà,"[7538669201578560786, 7539415204535258376, 754...",5
...,...,...,...,...
1479,20,người ta nói là anh đang thích nhỏ này nhỏ nào...,"[7538669201578560786, 7540258412328766728]",2
1480,20,nhiều người ngoài kia nói là anh tệ kệ mình nh...,"[7540258412328766728, 7541741554461936903]",2
1481,20,nói là anh đang thích nhỏ này nhỏ nào người ta...,"[7538669201578560786, 7540258412328766728]",2
1482,20,sánh bằng một kỳ nghỉ bằng máy bay phản lực và...,"[7538786096793079054, 7540973305436687624]",2


In [7]:
import pandas as pd

# dups có các cột: ['n', 'ngram', 'ids', 'id_count']
# đảm bảo ids là list sorted-unique để so sánh đúng
dups = dups.copy()
dups['ids'] = dups['ids'].map(lambda L: sorted(set(L)))

# Tạo khóa có thể hash để group
dups['ids_key'] = dups['ids'].map(tuple)

# Tìm n lớn nhất theo mỗi tổ hợp ids
dups['n_max'] = dups.groupby('ids_key')['n'].transform('max')

# Giữ lại *chỉ* các hàng có n == n_max (loại bỏ các n nhỏ hơn cho cùng tổ hợp ids)
dups_keep = dups[dups['n'] == dups['n_max']].drop(columns=['n_max']).copy()

In [8]:
dups_keep[dups_keep['n'] >= 3]

,n,ngram,ids,id_count,ids_key
296,3,chú ba minh,"[7540879122965318932, 7541095303907134741, 754...",4,"(7540879122965318932, 7541095303907134741, 754..."
297,3,mình nhắm mắt,"[7538669201578560786, 7540258412328766728, 754...",4,"(7538669201578560786, 7540258412328766728, 754..."
298,3,người đàn ông,"[7540557377603439880, 7540708188795014417, 754...",4,"(7540557377603439880, 7540708188795014417, 754..."
299,3,và cái kết,"[7540193333759528199, 7540361331124309266, 754...",4,"(7540193333759528199, 7540361331124309266, 754..."
304,3,ba minh người,"[7540708188795014417, 7540879122965318932, 754...",3,"(7540708188795014417, 7540879122965318932, 754..."
...,...,...,...,...,...
1479,20,người ta nói là anh đang thích nhỏ này nhỏ nào...,"[7538669201578560786, 7540258412328766728]",2,"(7538669201578560786, 7540258412328766728)"
1480,20,nhiều người ngoài kia nói là anh tệ kệ mình nh...,"[7540258412328766728, 7541741554461936903]",2,"(7540258412328766728, 7541741554461936903)"
1481,20,nói là anh đang thích nhỏ này nhỏ nào người ta...,"[7538669201578560786, 7540258412328766728]",2,"(7538669201578560786, 7540258412328766728)"
1482,20,sánh bằng một kỳ nghỉ bằng máy bay phản lực và...,"[7538786096793079054, 7540973305436687624]",2,"(7538786096793079054, 7540973305436687624)"


In [9]:
# Gom nhóm theo tổ hợp ids (và n – giờ n là n_max) để lấy danh sách n-gram
groups = (
    dups_keep
    .groupby(['ids_key', 'n'], as_index=False)
    .agg(
        ngrams=('ngram', lambda s: sorted(set(s))),   # danh sách n-gram (distinct)
        ngram_count=('ngram', 'nunique')              # số lượng n-gram
    )
)

# Phục hồi cột ids & id_count
groups['ids'] = groups['ids_key'].map(list)
groups['id_count'] = groups['ids'].str.len()

# Sắp xếp cho dễ xem: tổ hợp xuất hiện ở nhiều id hơn, n dài hơn, nhiều n-gram hơn
groups = groups.sort_values(
    ['id_count', 'n', 'ngram_count'], ascending=[False, False, False]
).reset_index(drop=True)

# Chọn cột cuối cùng
groups = groups[['n', 'ids', 'id_count', 'ngram_count', 'ngrams']]

print(groups.shape)

(212, 5)


In [12]:
import pandas as pd

g = groups.copy()
g['ids'] = g['ids'].map(lambda L: sorted(set(L)))
g['ids_set'] = g['ids'].map(frozenset)

# duyệt từ n lớn -> nhỏ; cùng n thì ưu tiên id_count, rồi ngram_count
g_sorted = g.sort_values(['n','id_count','ngram_count'],
                         ascending=[False, False, False]).reset_index(drop=True)

kept_idx = []
kept = []  # list of (n_kept, ids_set)

for i, row in g_sorted.iterrows():
    s = row['ids_set']
    ncur = int(row['n'])
    # xử lý cả trường hợp n lớn hơn và n bằng nhau (nên chỉ còn một nhóm cho mỗi id tại cùng n)
    conflict = any(((nkept > ncur) or (nkept == ncur)) and (not s.isdisjoint(ks))
                   for nkept, ks in kept)
    if conflict:
        continue
    kept_idx.append(i)
    kept.append((ncur, s))

groups_pruned = (
    g_sorted.loc[kept_idx]
    .drop(columns=['ids_set'])
    .sort_values(['id_count','n','ngram_count'], ascending=[False, False, False])
    .reset_index(drop=True)
)

print("Trước lọc:", len(groups), " | Sau lọc:", len(groups_pruned))

Trước lọc: 212  | Sau lọc: 29


In [11]:
groups_pruned.head(60)

,n,ids,id_count,ngram_count,ngrams
0,20,"[7538669201578560786, 7540258412328766728, 754...",3,25,[anh tệ kệ mình nhắm mắt yêu luôn người ta nói...
1,4,"[7540337679964114184, 7541085574015274241, 754...",3,1,[cảnh sát giao thông]
2,4,"[7540557377603439880, 7540708188795014417, 754...",3,1,[một người đàn ông]
3,20,"[7538786096793079054, 7540973305436687624]",2,6,[bằng một kỳ nghỉ bằng máy bay phản lực và nga...
4,6,"[7541252837200661768, 7541354087690931477]",2,1,[hợp luyện diễu binh diễu hành]
5,4,"[7539802409124629768, 7541363721034042642]",2,1,[thương má em hồng]
6,4,"[7540879122965318932, 7541095303907134741]",2,1,[chú ba minh người]
7,3,"[7537488053154204936, 7540999941137681682]",2,1,[mẹ mua cho]
8,3,"[7539415572929400072, 7540888882720541970]",2,1,[đâm thẳng vào]
9,3,"[7539855012164422920, 7539883369576811784]",2,1,[nó hông biết]
